# 準備

In [1]:
import datetime
print("Start Time: ", datetime.datetime.now())

Start Time:  2024-10-07 14:14:02.066766


## Google Drive に接続

In [2]:
from google.colab import drive
drive.mount('/content/drive')

##パス設定
full_path = '/content/drive/MyDrive/SenseWordsClustering/'

print("The Path is [", full_path, "]")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The Path is [ /content/drive/MyDrive/SenseWordsClustering/ ]


## ライブラリインポート

In [3]:
# 必要なパッケージのインストール（実行環境に応じて）
!pip install transformers torch
!pip install xlsxwriter

In [4]:
# 基本
import os

# データ処理
import pandas as pd
import xlsxwriter
import numpy as np

# ベクトル化
import torch
from transformers import BertModel, BertTokenizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

# クラスタリング
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture

## その他

In [5]:
# 乱数固定
random_seed = 42

# パス設定
input_dir = f'{full_path}input/'
output_dir = f'{full_path}output/'

# フォルダ作成
os.makedirs(output_dir, exist_ok=True)

# データ読み込み

In [6]:
# ファイルパスの設定
sense_word_file = '240930_感覚ワードリスト（一例）_v01.xlsx'
sense_word_path = f'{input_dir}{sense_word_file}'

# アップロードされたExcelファイルのシートを確認
xls = pd.ExcelFile(sense_word_path)

# 必要なシート「感覚ワード一例」を読み込む
df = pd.read_excel(sense_word_path, sheet_name='感覚ワード一例')
display(df)

,感覚ワードリスト
0,愛らしい
1,明るい
2,味わい深い
3,あたたかい
4,あっさり
...,...
85,優雅な
86,リフレッシュ
87,リラックス
88,レトロ


### 設定

In [7]:
### ----- ベクトル化手法を選択 -----
# vectorize_method = "bert"  #-> △
vectorize_method = "word2vec"  #-> ○
# vectorize_method = "tf-idf"  #-> ×


### ----- クラスタリング手法を選択 -----
clustering = "k-means"  #->○
# clustering = "agg-cluster"  #->○
# clustering = "dbscan"  #->×
# clustering = "gmm"  #->×


### ----- クラスタ数 -----
n_clusters = 10

# ベクトル化とクラスタリング

## ベクトル化

### BERTモデル

In [8]:
if vectorize_method == "bert":
    # BERTモデルとトークナイザーの読み込み
    model_name = 'cl-tohoku/bert-base-japanese'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # 感覚ワードをベクトル化する関数
    def bert_vectorizer(word):
        inputs = tokenizer(word, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # ベクトル化結果は[CLS]トークンに対応するものを使用
        return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

    # 感覚ワードリストの取得
    sense_word_list = df['感覚ワードリスト'].tolist()

    # 各感覚ワードをベクトル化
    vectorized_data = [bert_vectorizer(word) for word in sense_word_list]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This

### Word2Vec

In [9]:
if vectorize_method == "word2vec":
    # 各感覚ワードをリスト化 (単語を一つの文として扱う)
    sense_word_list = df['感覚ワードリスト'].tolist()
    sense_word_list_sentence = [[word] for word in sense_word_list]

    # Word2Vecモデルの学習 (意味的なベクトル化)
    model = Word2Vec(sentences=sense_word_list_sentence, vector_size=100, window=3, min_count=1, sg=0)

    # 各単語のベクトルを取得
    vectorized_data = np.array([model.wv[word] for word in df['感覚ワードリスト']])

### TF-IDF

In [10]:
if vectorize_method == "tf-idf":
    # 各感覚ワードをリスト化
    sense_word_list = df['感覚ワードリスト'].tolist()

    # ワードをTF-IDFでベクトル化
    vectorizer = TfidfVectorizer()
    vectorized_data = vectorizer.fit_transform(sense_word_list)

## クラスタリング

### KMeans

In [11]:
if clustering == "k-means":
    # KMeansでクラスタリング
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_seed)
    kmeans.fit(vectorized_data)

    # クラスタのラベルをデータフレームに追加
    df['クラスタ群'] = kmeans.labels_

### 階層的クラスタリング

In [12]:
if clustering == "agg-cluster":
    # 階層的クラスタリング
    clustering_model = AgglomerativeClustering(n_clusters=n_clusters)
    labels = clustering_model.fit_predict(vectorized_data)

    # クラスタのラベルをデータフレームに追加
    df['クラスタ群'] = labels

### DBSCAN

In [13]:
if clustering == "dbscan":
    # DBSCANクラスタリングの実行
    dbscan = DBSCAN(eps=0.5, min_samples=1)
    labels = dbscan.fit_predict(vectorized_data)

    # クラスタのラベルをデータフレームに追加
    df['クラスタ群'] = labels

### Gaussian Mixture Model

In [14]:
if clustering == "gmm":
    # Gaussian Mixture Model (GMM) を使ってクラスタリング
    gmm = GaussianMixture(n_components=n_clusters, random_state=random_seed)
    gmm.fit(vectorized_data)
    labels = gmm.predict(vectorized_data)

    # クラスタのラベルをデータフレームに追加
    df['クラスタ群'] = labels

##実行

In [15]:
# 結果をExcelに保存
output_file_path = f'{output_dir}クラスタリング結果_{vectorize_method}_{clustering}_{str(n_clusters)}.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='感覚ワードとクラスタ群', index=False)

    # クラスタごとの感覚ワードリストをまとめて別シートに保存
    cluster_group = {}
    for i in range(n_clusters):
        cluster_group[f'クラスタ {i}'] = df[df['クラスタ群'] == i]['感覚ワードリスト'].tolist()

    # クラスタごとの単語リストをデータフレームに変換して新しいシートに書き込む
    cluster_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in cluster_group.items()]))
    cluster_df.to_excel(writer, sheet_name='クラスタ別ワードリスト', index=False)

In [16]:
display(df)

,感覚ワードリスト,クラスタ群
0,愛らしい,5
1,明るい,0
2,味わい深い,2
3,あたたかい,4
4,あっさり,8
...,...,...
85,優雅な,5
86,リフレッシュ,1
87,リラックス,3
88,レトロ,3
